<font size="+3"><center>
    CCATp Zilinx zcu111 RFSoC Initial Setup Procedure
</center></font>

<center><b>James Burgoyne</b></center>
<center>jburgoyne@phas.ubc.ca</center>
<center>August 2022</center>

**Development system:**
- Mac OS 12.5 on a Macbook Pro early 2015.
- Python 3.7.11

# SD Card Image

The board is being run off of a micro SD card.  
The image for the micro SD card can be downloaded from: http://www.pynq.io/board.html/  
We are using version 2.7.  
Instructions for writing the image: https://pynq.readthedocs.io/en/latest/appendix/sdcard.html  
To write the image to the card you'll need a card reader.  
Launch Disk Utility and note the disk number of the card (in my case disk2).  
From Disk Utility unmount (not eject) the card.  
Image the card:  
`sudo dd bs=1m if=zcu111_v2.7.0.img of=/dev/rdisk2`  
It takes about 10 minutes to finish.

# Board Interface

Interface connection: Ethernet via USB.  
Using Plugable USB-C to ethernet adapter: https://plugable.com/products/usbc-e2500  
This adapter allows for jumbo frames.

The adapter was given a manual IP4 address of 192.168.2.80 on the development system (Network Preferences -> USB 10/100/1G/2.5G LAN -> Configure IPv4: Manually -> IP Adress: 192.168.2.80, Subnet Mask: 255.255.255.0

The 2x2 board ethernet connection lives at 192.168.2.99 and can be accessed through SSH or Jupyter with:  
**username: xilinx   
password: xilinx**

# Redis

The Redis server on the development system is version 5.0.3.  
The redis.conf file was edited to change the IP binding: bind 192.168.2.80  

The 2x2 board did not have Redis installed in the default 2.7 image. 
Version 5.0.14 (the lastest 5.x version) was downloaded from: https://redis.io/download/#redis-downloads  
It was copied to the 2x2:  
`scp redis-5.0.14.tar.gz xilinx@192.168.2.99:/home/xilinx/`  
The 2x2 is SSH accessed via:  
`ssh xilinx@192.168.2.99`  
Unpack the tar.gz file:  
`tar -xvf redis-5.0.14.tar.gz`  
switch to the new directory:  
`cd redis-5.0.14`  
and install Redis with:  
`make` (should take about 5-10 minutes)  
and then  
`sudo make install`  
Check:  
`redis-server --version`

# Redis-py

The Redis python module is also necessary.  
https://pypi.org/project/redis/  
On the development system:  
`scp redis-4.3.3.tar.gz xilinx@192.168.2.99:/home/xilinx`  
and then on the 2x2:  
`tar -xvf redis-4.3.3.tar.gz`  
`cd redis-4.3.3/`  
`sudo python setup.py install`  
Note the final return message:  
_error: Could not find suitable distribution for Requirement.parse('packaging>=20.4')_  
This may have to be dealt with at some point, but things seem to work without it.

# Hive

The CCATpHive software was installed on the 2x2:  
On the development system, zip up the CCATpHive folder:  
`tar -czf CCATpHive.tgz CCATpHive`  
and copy to the 2x2:  
`scp CCATpHive.tgz xilinx@192.168.2.99:/home/xilinx/CCATpHive.tgz`  
then switch to the 2x2:  
`ssh xilinx@192.168.2.99`    
and unpack it:  
`tar -xvf CCATpHive.tgz`  
A few unecessary files were deleted:  
`cd CCATpHive`  
`rm -r __pycache__`  
`rm dump.rdb`  
`rm tmp/*`  
`rm logs/*`  
Finally, the config file was modified:  
`nano _cfg_board.py`  
and the host changed to '192.168.2.80' (ctrl-o to save, ctrl-x to quit).  
Back on the development system:  
_cfg_queen.py was modified and the host changed to 192.168.2.80.  